In [176]:
import torch
import numpy as np
import cv2
device = 'cuda:0'

# 関数定義
def equirectangular_to_spherical_coords(img_width, img_height, device='cpu'):
    theta = torch.linspace(0, np.pi, img_height, device=device)  # 0からπ
    phi = torch.linspace(0, 2 * np.pi, img_width, device=device)  # 0から2π
    phi_grid, theta_grid = torch.meshgrid(phi, theta, indexing="ij")
    return torch.stack([theta_grid, phi_grid, torch.ones_like(theta_grid)], dim=-1)

In [177]:
img_width, img_height = 1024, 512
coords = equirectangular_to_spherical_coords(img_width, img_height, device)

In [178]:
# テスト関数
def test_equirectangular_to_spherical_coords():
    # 画像の幅と高さ
    img_width, img_height = 360, 180
    
    # デバイス設定
    device = 'cpu'
    
    # 関数から座標を取得
    coords = equirectangular_to_spherical_coords(img_width, img_height, device)
    
    # 仰角θのテスト
    assert torch.allclose(coords[:, :, 0].min(), torch.tensor(0.0, device=device)), "Minimum theta should be 0"
    assert torch.allclose(coords[:, :, 0].max(), torch.tensor(np.pi, device=device)), "Maximum theta should be pi"
    
    # 方位角φのテスト
    assert torch.allclose(coords[:, :, 1].min(), torch.tensor(0.0, device=device)), "Minimum phi should be 0"
    assert torch.allclose(coords[:, :, 1].max(), torch.tensor(2 * np.pi, device=device)), "Maximum phi should be 2*pi"
    
    # すべてのテストが成功した場合のメッセージ
    print("All tests passed!")

# テスト関数を呼び出し
test_equirectangular_to_spherical_coords()

All tests passed!


In [179]:
def adjust_spherical_coords(spherical_coords):
    # θ (theta) と φ (phi) を抽出する
    theta, phi, _ = spherical_coords[..., 0], spherical_coords[..., 1], spherical_coords[..., 2]

    # φの値に基づいてθを調整する
    theta_adjusted = torch.where((phi < torch.pi), theta + torch.pi / 4, theta + torch.pi / 4)

    # θがπを超える場合の調整
    theta_adjusted = torch.where(theta_adjusted < torch.pi, theta_adjusted, 2 * torch.pi - theta_adjusted)
    phi_adjusted = torch.where(theta_adjusted < torch.pi, phi, 2 * torch.pi - phi)

    # θが0未満の場合の調整
    theta_adjusted = torch.where(theta_adjusted >= 0, theta_adjusted, -theta_adjusted)
    phi_adjusted = torch.where(theta_adjusted >= 0, phi_adjusted, 2 * torch.pi - phi_adjusted)

    # 更新された値で新しい座標配列を構築
    new_coords = torch.stack([theta_adjusted, phi_adjusted, torch.ones_like(theta_adjusted)], dim=-1)
    
    return new_coords


In [180]:
new_coords = adjust_spherical_coords(coords)

In [181]:
print(new_coords)

tensor([[[7.8540e-01, 0.0000e+00, 1.0000e+00],
         [7.9155e-01, 0.0000e+00, 1.0000e+00],
         [7.9769e-01, 0.0000e+00, 1.0000e+00],
         ...,
         [2.3685e+00, 0.0000e+00, 1.0000e+00],
         [2.3623e+00, 0.0000e+00, 1.0000e+00],
         [2.3562e+00, 0.0000e+00, 1.0000e+00]],

        [[7.8540e-01, 6.1419e-03, 1.0000e+00],
         [7.9155e-01, 6.1419e-03, 1.0000e+00],
         [7.9769e-01, 6.1419e-03, 1.0000e+00],
         ...,
         [2.3685e+00, 6.1419e-03, 1.0000e+00],
         [2.3623e+00, 6.1419e-03, 1.0000e+00],
         [2.3562e+00, 6.1419e-03, 1.0000e+00]],

        [[7.8540e-01, 1.2284e-02, 1.0000e+00],
         [7.9155e-01, 1.2284e-02, 1.0000e+00],
         [7.9769e-01, 1.2284e-02, 1.0000e+00],
         ...,
         [2.3685e+00, 1.2284e-02, 1.0000e+00],
         [2.3623e+00, 1.2284e-02, 1.0000e+00],
         [2.3562e+00, 1.2284e-02, 1.0000e+00]],

        ...,

        [[7.8540e-01, 6.2709e+00, 1.0000e+00],
         [7.9155e-01, 6.2709e+00, 1.0000e+00]

In [182]:
def spherical_to_equirectangular_coords(spherical_coords, img_width, img_height):
    theta, phi, _ = spherical_coords[..., 0], spherical_coords[..., 1], spherical_coords[..., 2]

    # 画像の高さと幅に基づいてピクセル座標を計算
    x = (phi / (2 * torch.pi)) * img_width  # 0 <= φ < 2π の範囲で img_width にマッピング
    y = (theta / torch.pi) * img_height     # 0 <= θ < π の範囲で img_height にマッピング

    # 座標が整数値である必要があるため、最も近い整数に丸める
    x = torch.round(x).long() % img_width   # img_width を超えないようにする
    y = torch.round(y).long() % img_height  # img_height を超えないようにする

    return x, y

In [183]:
def create_3dmap_from_size_torch(img_w, img_h, device):
    h = torch.linspace(-np.pi/2, np.pi/2, img_h, device=device)
    w = torch.linspace(-np.pi, np.pi, img_w, device=device)
    
    h += (np.pi/2) / img_h
    w += np.pi / img_w
    
    theta, phi = torch.meshgrid(w, h, indexing="ij")
    
    x = torch.cos(phi) * torch.cos(theta)
    y = torch.cos(phi) * torch.sin(theta)
    z = torch.sin(phi)
    
    return x, y, z

In [184]:
import torch
import numpy as np

def equirectangular_to_spherical_coords(img_width, img_height, device='cpu'):
    theta = torch.linspace(0, np.pi, img_height, device=device)  # 0からπ
    phi = torch.linspace(0, 2 * np.pi, img_width, device=device)  # 0から2π
    phi_grid, theta_grid = torch.meshgrid(phi, theta, indexing="xy")
    return torch.stack([theta_grid, phi_grid, torch.ones_like(theta_grid)], dim=-1)

def adjust_spherical_coords(spherical_coords):
    theta, phi, _ = spherical_coords[..., 0], spherical_coords[..., 1], spherical_coords[..., 2]

    theta_adjusted = torch.where((phi < torch.pi), theta + torch.pi / 4, theta - torch.pi / 4)
    phi_adjusted = torch.where(theta_adjusted < torch.pi, phi, 2 * torch.pi - phi)
    theta_adjusted = torch.where(theta_adjusted < torch.pi, theta_adjusted, 2 * torch.pi - theta_adjusted)
    phi_adjusted = torch.where(theta_adjusted >= 0, phi_adjusted, 2 * torch.pi - phi_adjusted)
    theta_adjusted = torch.where(theta_adjusted >= 0, theta_adjusted, -theta_adjusted)
    return torch.stack([theta_adjusted, phi_adjusted, torch.ones_like(theta_adjusted)], dim=-1)


def spherical_to_equirectangular_coords(spherical_coords, img_width, img_height):
    theta, phi, _ = spherical_coords[..., 0], spherical_coords[..., 1], spherical_coords[..., 2]
    x = (phi / (2 * torch.pi)) * img_width
    y = (theta / torch.pi) * img_height
    x = torch.round(x).long() % img_width
    y = torch.round(y).long() % img_height
    return x, y

def transform_equirectangular_image(img):
    img_height, img_width = img.shape[:2]
    spherical_coords = equirectangular_to_spherical_coords(img_width, img_height, device)
    adjusted_coords = adjust_spherical_coords(spherical_coords)
    x, y = spherical_to_equirectangular_coords(adjusted_coords, img_width, img_height)
    #x, y = spherical_to_equirectangular_coords(spherical_coords, img_width, img_height)

    # 新しい画像を作成
    transformed_img = torch.zeros_like(img)
    for i in range(img_height):
        for j in range(img_width):
            y_idx = min(max(y[i, j], 0), img_height - 1)
            x_idx = min(max(x[i, j], 0), img_width - 1)
            transformed_img[y_idx, x_idx] = img[i, j]
    return transformed_img


In [185]:
img = cv2.imread("./data/data_100/Room/0/O.png")
#cv2.imshow("Frame", img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
img = torch.from_numpy(img)
print(img.shape)

torch.Size([512, 1024, 3])


In [186]:
img2 = transform_equirectangular_image(img)

In [187]:
if img2.is_cuda:
    img2 = img2.cpu()

# Tensor を NumPy 配列に変換
img_numpy = img2.numpy()

# データ型が float の場合、0-255 に正規化して uint8 に変換
if img_numpy.dtype == np.float32 or img_numpy.dtype == np.float64:
    img_numpy = (img_numpy * 255).astype(np.uint8)

# チャネルの順序が (C, H, W) の場合、(H, W, C) に変更し、BGR に変換
if img_numpy.shape[0] == 3:
    img_numpy = img_numpy.transpose(1, 2, 0)  # C, H, W -> H, W, C
    img_numpy = cv2.cvtColor(img_numpy, cv2.COLOR_RGB2BGR)
cv2.imshow("Frame", img_numpy)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [188]:
img = cv2.imread("./data/data_100/Room/0/O.png")
img_height, img_width = img.shape[:2]
spherical_coords = equirectangular_to_spherical_coords(img_width, img_height, device)
adjusted_coords = adjust_spherical_coords(spherical_coords)
x, y = spherical_to_equirectangular_coords(adjusted_coords, img_width, img_height)

In [189]:
adjusted_coords.shape

torch.Size([512, 1024, 3])

In [190]:
spherical_coords

tensor([[[0.0000e+00, 0.0000e+00, 1.0000e+00],
         [0.0000e+00, 6.1419e-03, 1.0000e+00],
         [0.0000e+00, 1.2284e-02, 1.0000e+00],
         ...,
         [0.0000e+00, 6.2709e+00, 1.0000e+00],
         [0.0000e+00, 6.2770e+00, 1.0000e+00],
         [0.0000e+00, 6.2832e+00, 1.0000e+00]],

        [[6.1479e-03, 0.0000e+00, 1.0000e+00],
         [6.1479e-03, 6.1419e-03, 1.0000e+00],
         [6.1479e-03, 1.2284e-02, 1.0000e+00],
         ...,
         [6.1479e-03, 6.2709e+00, 1.0000e+00],
         [6.1479e-03, 6.2770e+00, 1.0000e+00],
         [6.1479e-03, 6.2832e+00, 1.0000e+00]],

        [[1.2296e-02, 0.0000e+00, 1.0000e+00],
         [1.2296e-02, 6.1419e-03, 1.0000e+00],
         [1.2296e-02, 1.2284e-02, 1.0000e+00],
         ...,
         [1.2296e-02, 6.2709e+00, 1.0000e+00],
         [1.2296e-02, 6.2770e+00, 1.0000e+00],
         [1.2296e-02, 6.2832e+00, 1.0000e+00]],

        ...,

        [[3.1293e+00, 0.0000e+00, 1.0000e+00],
         [3.1293e+00, 6.1419e-03, 1.0000e+00]

In [191]:
adjusted_coords

tensor([[[7.8540e-01, 0.0000e+00, 1.0000e+00],
         [7.8540e-01, 6.1419e-03, 1.0000e+00],
         [7.8540e-01, 1.2284e-02, 1.0000e+00],
         ...,
         [7.8540e-01, 1.2284e-02, 1.0000e+00],
         [7.8540e-01, 6.1421e-03, 1.0000e+00],
         [7.8540e-01, 0.0000e+00, 1.0000e+00]],

        [[7.9155e-01, 0.0000e+00, 1.0000e+00],
         [7.9155e-01, 6.1419e-03, 1.0000e+00],
         [7.9155e-01, 1.2284e-02, 1.0000e+00],
         ...,
         [7.7925e-01, 1.2284e-02, 1.0000e+00],
         [7.7925e-01, 6.1421e-03, 1.0000e+00],
         [7.7925e-01, 0.0000e+00, 1.0000e+00]],

        [[7.9769e-01, 0.0000e+00, 1.0000e+00],
         [7.9769e-01, 6.1419e-03, 1.0000e+00],
         [7.9769e-01, 1.2284e-02, 1.0000e+00],
         ...,
         [7.7310e-01, 1.2284e-02, 1.0000e+00],
         [7.7310e-01, 6.1421e-03, 1.0000e+00],
         [7.7310e-01, 0.0000e+00, 1.0000e+00]],

        ...,

        [[2.3685e+00, 6.2832e+00, 1.0000e+00],
         [2.3685e+00, 6.2770e+00, 1.0000e+00]

In [249]:
img = cv2.imread("./data/data_100/Room/0/O.png")
h,w = img.shape[:2]
 
w = int(w/2)
h = int(h/2)

theta, phi = np.meshgrid(np.arange(-np.pi/2, np.pi/2, np.pi/w),
                         np.arange(-np.pi/2, np.pi/2, np.pi/h))
 
x = np.cos(phi)*np.cos(theta)
y = np.cos(phi)*np.sin(theta)
z = np.sin(phi)
 
rot = np.pi/2
     
xx = x*np.cos(rot) + z*np.sin(rot)
yy = y
zz = -x*np.sin(rot) + z*np.cos(rot)
     
phi = np.arcsin(zz)
theta = np.arcsin(yy / np.cos(phi))
     
Y = (2 * phi / np.pi) * h + h
X = (theta / np.pi) * w + w
     
out = cv2.remap(img, X.astype(np.float32), Y.astype(np.float32), cv2.INTER_LINEAR)
cv2.imwrite("dst.jpg", out)

/home/imonalc/anaconda3/envs/keypoints/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in arcsin


True

In [261]:
import cv2
import numpy as np

# 画像の読み込み
img = cv2.imread("./data/data_100/Room/0/O.png")
img = cv2.imread("./data/example/farm2/R.png")
h, w = img.shape[:2]

w_half = int(w / 2)
h_half = int(h / 2)

# メッシュグリッドの生成
theta, phi = np.meshgrid(np.linspace(-np.pi, np.pi, w_half*2),
                         np.linspace(-np.pi/2, np.pi/2, h_half*2))


# 球面座標
x = np.cos(phi) * np.cos(theta)
y = np.cos(phi) * np.sin(theta)
z = np.sin(phi)

# z軸周りにπ/2ラジアン回転
rot = np.pi / 2
xx = x * np.cos(rot) + z * np.sin(rot)
yy = y
zz = -x * np.sin(rot) + z * np.cos(rot)

# 逆球面座標変換
phi = np.arcsin(zz)
theta = np.arctan2(yy, xx)

# 画像座標へのマッピング
Y = 2*phi / np.pi * h_half + h_half
X = theta / np.pi * w_half + w_half

# 画像のリマッピングと保存
out = cv2.remap(img, X.astype(np.float32), Y.astype(np.float32), cv2.INTER_LINEAR, borderMode=cv2.BORDER_WRAP)
cv2.imwrite("dst.png", out)


True